In [ ]:
import sys
import os
import shutil
import numpy as np
import yaml
sys.path.append('/home/welling/git/tensorflow_apps/src')
from brainroller.traceneighbors import UsefulVtx
from brainroller.writegeom import writeVtkPolypoints
from cnn.constants import RAD_PIXELS
import spilltree3D

In [ ]:
traceFile = '/tmp/useful_trace_neighborhoods.pkl'
skipFile = '/tmp/skips.txt'


In [ ]:
with open(traceFile, 'rb') as pklF:
    with open(skipFile, 'rb') as skipF:
        usefulVtxDict = UsefulVtx.load(pklF, 10000, skipF)


In [ ]:
spT = spilltree3D.SpTree(usefulVtxDict.values())


In [ ]:
assert spT.getNDim() == 3, 'spilltree3D was compiled incorrectly'

In [ ]:
fishCoreXSize = 1024
fishCoreYSize = 1024
fishCoreZSize = 4900
fishCoreXOffset = 4750. - (fishCoreXSize/2)
fishCoreYOffset = 2150. - (fishCoreYSize/2)
fishCoreZOffset = 4000

fishCoreBlockSize = 1024  # how many Z slices to actually load

scan_sz_V = np.asarray([41., 41., 41.])

In [ ]:
root_dir = '/home/welling/sshfshook'
fish_cube_dir = 'pylon2_rescue/fish_cube_links'
output_root = '/home/welling/git/tensorflow_apps/src/joel'

In [ ]:
def write_all_verts_vtk(fname, sample_id, ctrCoords, usefulVtxDict, spill_tree, scan_sz_V, offsetV):
    if sample_id is None:
        ctrX, ctrY, ctrZ = ctrCoords
        ctrVtx = UsefulVtx(None, None, None, ctrX, ctrY, ctrZ)
    else:
        ctrVtx = usefulVtxDict[sample_id]
    max_sum_sqr = 0.25 * np.sum(scan_sz_V * scan_sz_V)
    sameObjLocL = []
    allLocL = []
    for uv, sepsqr in spill_tree.findApproxKNearest(1000, ctrVtx):
        allLocL.append(uv)
        if uv.objId == ctrVtx.objId:
            sameObjLocL.append(uv)
        if sepsqr > max_sum_sqr:
            break
    else:
        raise RuntimeError('More than 1000 verts in range!')
    print('%d nearby verts, %d in same object' % (len(allLocL), len(sameObjLocL)))
    realV = np.asarray(ctrVtx.realCoords)
    lclV = realV - offsetV

    x_off, y_off, z_off = offsetV + lclV
    shift1 = [-x_off, -y_off, -z_off]
    x2_off, y2_off, z2_off = - 0.5 * scan_sz_V
    shift2 = [-x2_off, -y2_off, -z2_off]
    writeVtkPolypoints(fname, allLocL, shift1=shift1, shift2=shift2, sig=[1.0, -1.0, 1.0])

def test_calcs(sample_id, empty=False):
    if empty:
        with open(os.path.join(root_dir, fish_cube_dir, 'empty_41_%d.yaml' % sample_id)) as f:
            infoD = yaml.safe_load(f)
    else:
        with open(os.path.join(root_dir, fish_cube_dir, 'block_41_%d.yaml' % sample_id)) as f:
            infoD = yaml.safe_load(f)
    ctrCoords = np.asarray(infoD['realCoords'])
    offsetV = np.asarray([int(np.floor(v)) for v in [fishCoreXOffset, fishCoreYOffset, fishCoreZOffset]])
    scanCoords = (ctrCoords - offsetV) - np.floor(0.5 * scan_sz_V)

    # Walk forward through the full dataset as necessary to get the target into the loaded part of the data
    nSkip = 0
    stepZ = fishCoreBlockSize // 2
    while (scanCoords[2] - nSkip) + scan_sz_V[2] > fishCoreBlockSize:
        nSkip += stepZ
    scanCoords[2] -= nSkip

    print('SCAN CONTROL PARAMETERS:')
    print('--data_block_dims=%d,%d,%d' % (fishCoreXSize, fishCoreYSize, fishCoreBlockSize))
    print('--data_block_offset=%d' % (nSkip * fishCoreXSize * fishCoreYSize))
    print('--scan_start=%s' % ','.join([str(int(v)) for v in np.floor(scanCoords)]))
    print('--scan_size=%s' % ','.join([str(int(v)) for v in scan_sz_V]))
    rslt_dir = os.path.join(output_root, 'test_%d' % sample_id)
    os.makedirs(rslt_dir, exist_ok=True)
    print('WRITING DATA TO: %s' % rslt_dir)
    for fname in infoD['bovFiles'] + [infoD['traceFile']]:
        shutil.copy(os.path.join(root_dir, fish_cube_dir, fname), rslt_dir)
    if empty:
        write_all_verts_vtk(os.path.join(rslt_dir, 'known_verts_%d.vtk' % sample_id),
                            None, ctrCoords, usefulVtxDict, spT, scan_sz_V, offsetV)
    else:
        write_all_verts_vtk(os.path.join(rslt_dir, 'known_verts_%d.vtk' % sample_id),
                            sample_id, ctrCoords, usefulVtxDict, spT, scan_sz_V, offsetV)
    print('\n')

In [ ]:
test_calcs(1010391)
test_calcs(994, empty=True)
#test_calcs(983792)
#test_calcs(2453393)
  

In [ ]:
help(UsefulVtx)